# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd


In [3]:
#Your code here; create the connection and cursor

conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [4]:
#Your code here; use a subquery. No join will be necessary.

cur.execute("""
            select customerNumber,
           contactLastName,
           contactFirstName
           from customers
           join orders 
           using(customerNumber)
           where orderDate = '2003-01-31';
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [5]:
#Your code here; use a subquery. No join will be necessary.

cur.execute("""
            SELECT customerNumber, contactLastName, contactFirstName
            FROM CUSTOMERS
            WHERE customerNumber in (SELECT customerNumber FROM orders WHERE orderDate = '2003-01-31')
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [9]:
#Your code here

cur.execute("""
            SELECT t1.productCode, t1.productName, sum(t2.quantityOrdered) Total_Sold
            FROM products t1
            INNER JOIN orderDetails t2
            on t1.productCode = t2.productCode
            GROUP BY t1.productCode
            ORDER BY Total_Sold DESC
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)

,productCode,productName,Total_Sold
0,S18_3232,1992 Ferrari 360 Spider red,1808
1,S18_1342,1937 Lincoln Berline,1111
2,S700_4002,American Airlines: MD-11S,1085
3,S18_3856,1941 Chevrolet Special Deluxe Cabriolet,1076
4,S50_1341,1930 Buick Marquette Phaeton,1074
5,S18_4600,1940s Ford truck,1061
6,S10_1678,1969 Harley Davidson Ultimate Chopper,1057
7,S12_4473,1957 Chevy Pickup,1056
8,S18_2319,1964 Mercedes Tour Bus,1053
9,S24_3856,1956 Porsche 356A Coupe,1052


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

,productcode,total_customer
0,S18_3232,10615
1,S24_2841,8964
2,S24_3420,8750
3,S18_1662,8664
4,S72_1253,8547
5,S18_3856,8425
6,S700_2047,8416
7,S18_3029,8095
8,S18_2957,8037
9,S10_4757,8000


In [64]:
#Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

cur.execute("""
            SELECT bt.productCode, count(customerNumber) total_customer
            FROM (
                SELECT DISTINCT t1.productcode, t4.customerNumber
                FROM products t1
                JOIN orderdetails t2 ON t1.productCode = t2.productcode
                JOIN orders t3 ON t2.orderNumber = t3.orderNumber
                JOIN customers t4 ON t3.customernumber = t4.customerNumber
                ) bt
            GROUP BY bt.productCode
            ORDER BY total_customer DESC
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(20)

,productcode,total_customer
0,S18_3232,40
1,S10_1949,27
2,S10_4757,27
3,S18_2957,27
4,S72_1253,27
5,S10_1678,26
6,S10_2016,26
7,S18_1662,26
8,S18_1984,26
9,S18_2949,26


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [77]:
#Your code here

cur.execute("""
                SELECT t1.productcode, COUNT(DISTINCT t4.customerNumber) total_cust
                FROM products t1
                JOIN orderdetails t2 ON t1.productCode = t2.productcode
                JOIN orders t3 ON t2.orderNumber = t3.orderNumber
                JOIN customers t4 ON t3.customernumber = t4.customerNumber
                GROUP BY t1.productCode
                HAVING total_cust <20
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)

,productCode,total_cust
0,S24_2766,18
1,S24_2840,19
2,S24_2887,18
3,S24_3191,18
4,S24_3432,18


In [80]:
#Your code here

cur.execute("""
            SELECT e.employeeNumber, e.lastName, e.firstname, of.city, of.officeCode
            FROM employees e
            JOIN offices of USING(officeCode)
            JOIN customers c ON e.employeenumber = c.salesrepemployeenumber
            JOIN orders o USING(customerNumber)
            JOIN orderdetails od USING(orderNumber)
            JOIN products p USING(productCode)
            WHERE p.productCode IN
                (
                SELECT t1.productcode
                FROM products t1
                JOIN orderdetails t2 ON t1.productCode = t2.productcode
                JOIN orders t3 ON t2.orderNumber = t3.orderNumber
                JOIN customers t4 ON t3.customernumber = t4.customerNumber
                GROUP BY t1.productCode
                HAVING COUNT(DISTINCT t4.customerNumber) <20
                )
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)

,employeeNumber,lastName,firstName,city,officeCode
0,1501,Bott,Larry,London,7
1,1166,Thompson,Leslie,San Francisco,1
2,1612,Marsh,Peter,Sydney,6
3,1611,Fixter,Andy,Sydney,6
4,1401,Castillo,Pamela,Paris,4
5,1323,Vanauf,George,NYC,3
6,1165,Jennings,Leslie,San Francisco,1
7,1216,Patterson,Steve,Boston,2
8,1286,Tseng,Foon Yue,NYC,3
9,1370,Hernandez,Gerard,Paris,4


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [91]:
#Your code here

cur.execute("""
            SELECT e.employeeNumber, e.firstname, e.lastname, count(creditLimit) num_cust, avg(creditLimit) avg_limit
            FROM employees e
            JOIN customers c ON e.employeenumber = c.salesrepemployeenumber
            GROUP BY e.employeeNumber
            HAVING avg_limit > 15000
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)

,employeeNumber,firstName,lastName,num_cust,avg_limit
0,1165,Leslie,Jennings,6,100433.333333
1,1166,Leslie,Thompson,6,65266.666667
2,1188,Julie,Firrelli,6,73916.666667
3,1216,Steve,Patterson,6,81533.333333
4,1286,Foon Yue,Tseng,7,66614.285714
5,1323,George,Vanauf,8,80887.500000
6,1337,Loui,Bondur,6,86233.333333
7,1370,Gerard,Hernandez,7,91785.714286
8,1401,Pamela,Castillo,10,81340.000000
9,1501,Larry,Bott,8,91187.500000


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!